In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras import layers
from IPython.display import Image as img
import random
from __future__ import absolute_import, division, print_function

from tensorflow import keras

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pathlib
from google.colab import drive
sess = tf.InteractiveSession()
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://github.com/epicrunze/Stars.git

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'Stars' already exists and is not an empty directory.


In [3]:
#Getting all image paths
imgdata_all = pathlib.Path("Stars/TestData")
imagePaths = list(imgdata_all.glob('*/*'))
imagePaths = [str(path) for path in imagePaths]

random.shuffle(imagePaths)

image_count = len(imagePaths)
print(imagePaths)


#assigning label names
label_names = sorted(item.name for item in imgdata_all.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
#assigning label indicies to data
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in imagePaths]


['Stars/TestData/leo/94.png', 'Stars/TestData/orion/93 (6).png', 'Stars/TestData/ursa major/151 (3).png', 'Stars/TestData/ursa major/150 (2).png', 'Stars/TestData/leo/24 (3).png', 'Stars/TestData/ursa major/3 (7).png', 'Stars/TestData/orion/175 (2).png', 'Stars/TestData/gemini/50 (8).png', 'Stars/TestData/gemini/186 (3).png', 'Stars/TestData/orion/43 (5).png', 'Stars/TestData/gemini/70 (6).png', 'Stars/TestData/leo/303 (4).png', 'Stars/TestData/cassiopeia/32 (4).png', 'Stars/TestData/gemini/327 (3).png', 'Stars/TestData/leo/321.png', 'Stars/TestData/ursa major/152 (4).png', 'Stars/TestData/orion/350 (3).png', 'Stars/TestData/orion/81 (6).png', 'Stars/TestData/leo/173.png', 'Stars/TestData/orion/167 (7).png', 'Stars/TestData/ursa major/110 (1).png', 'Stars/TestData/gemini/269 (3).png', 'Stars/TestData/ursa major/304 (1).png', 'Stars/TestData/orion/71 (7).png', 'Stars/TestData/orion/249 (3).png', 'Stars/TestData/orion/341 (4).png', 'Stars/TestData/ursa major/32 (5).png', 'Stars/TestData/

In [4]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def preprocess_image(image):
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image, [256, 256])
    image = tf.math.divide(image, 255.0) # normalize to [0,1] range
    return image
#making tensorflow datasets from images
path_ds = tf.data.Dataset.from_tensor_slices(imagePaths)
image_ds = path_ds.map(load_and_preprocess_image)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

print(image_label_ds)
print('image shape: ', image_label_ds.output_shapes[0])
print('label shape: ', image_label_ds.output_shapes[1])
print('types: ', image_label_ds.output_types)
print()

image_label_ds = image_label_ds.batch(32)
image_label_ds = image_label_ds.repeat()
index_to_label = dict((index, name) for index,name in enumerate(label_names))

W0614 16:00:49.904720 139909664003968 deprecation.py:323] From <ipython-input-4-96080097421c>:18: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
W0614 16:00:49.907049 139909664003968 deprecation.py:323] From <ipython-input-4-96080097421c>:20: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.


<DatasetV1Adapter shapes: ((256, 256, 1), ()), types: (tf.float32, tf.int64)>
image shape:  (256, 256, 1)
label shape:  ()
types:  (tf.float32, tf.int64)



In [5]:

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform


model = load_model("gdrive/My Drive/School/Gr 12/Space/Constellation Project/SavedModel1.h5")
testimagePaths = ["Stars/geminitest.png"]
test_path_ds = tf.data.Dataset.from_tensor_slices(testimagePaths)
test_image_ds = test_path_ds.map(load_and_preprocess_image)
TestSet = np.full((0, 256, 256, 1), -1)

test_iterator = test_image_ds.make_one_shot_iterator()
test_next_element = test_iterator.get_next()


for i in range(len(testimagePaths)):
    a = np.expand_dims(test_next_element.eval(session=sess), axis=0)
    TestSet = np.append(TestSet, a, axis=0)
    
print(TestSet.shape)

W0614 16:01:26.887104 139909664003968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0614 16:01:26.888947 139909664003968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0614 16:01:26.890692 139909664003968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

(1, 256, 256, 1)


In [0]:
results = model.predict_classes(TestSet)

In [7]:
print(index_to_label)
for i in range(len(results)):
    print("The Result is: " + str(index_to_label[results[i]]))

{0: 'cassiopeia', 1: 'gemini', 2: 'leo', 3: 'orion', 4: 'ursa major'}
The Result is: gemini
